# In-Class Exercise: Stock Class (in_class_4_str.py)

### Instructions

Starting with the file **`in_class_4_str.py`**, located in the **`python_files`** folder in the course repository, complete the following tasks:

---
### Add Methods to the `Stock` Class

#### a) `get_data`
- Use a free stock data API (e.g., the Python library `yfinance`) to download historical price data.
- The method should accept user-defined stock symbol, starting date, and ending date in ISO format.
- If dates are not provided, default to the last one year of data as of today’s close.
- Store the data in a **pandas DataFrame**.
- Set the DataFrame index to the **date**, and convert it to a pandas **DatetimeIndex**.

---

#### b) `calc_returns`
- A helper method called automatically by `get_data`.
- Add the following columns to the DataFrame:
  - **`change`** – the difference between the close-to-close price relative to the previous day’s close.
  - **`instant_return`** – the daily instantaneous rate of return computed as `np.log([closing_price]).diff().round(4)`.

---

#### c) `plot_return_dist`
- Plot a well-formatted **histogram** of the instantaneous returns.

---

#### d) `plot_performance`
- Plot a **line graph** showing the stock’s performance over the collected data range, expressed as a percent gain/loss relative to the starting value.

---

### Test Your Class

1. Instantiate a test `Stock` object.
2. Access the `.data` attribute to view the DataFrame.
3. Generate both plots:
   - Instantaneous Return Distribution  
   - Performance (% Gain/Loss)

---
**Deliverable:**  
A Jupyter Notebook (Markdown formatted) showing the code implementation, method testing, and visual outputs.


In [13]:
# in_class_4_str.py
# Professor-style implementation for fetching data, computing returns, and plotting.
# Requires: yfinance, pandas, numpy, matplotlib, seaborn (theme optional)
# ===== Imports =====
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sb
import yfinance as yf   # moved to top so all imports are consolidated


In [15]:
# ===== Visualization Defaults =====
sb.set_theme(style="whitegrid", context="talk")

# ===== Constants =====
DEFAULT_START = dt.date.isoformat(dt.date.today() - dt.timedelta(365))
DEFAULT_END = dt.date.isoformat(dt.date.today())

In [24]:
# in_class_4_str.py
# Professor-style implementation for fetching data, computing returns, and plotting.

# ===== Imports =====
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sb
import yfinance as yf   # moved to top so all imports are consolidated




# ===== Class Definition =====
class Stock:
    def __init__(self, symbol: str, start: str = DEFAULT_START, end: str = DEFAULT_END):
        """
        Parameters
        ----------
        symbol : str
            Ticker string, e.g., 'AAPL'
        start : str
            ISO date 'YYYY-MM-DD' (defaults to 1 year ago)
        end : str
            ISO date 'YYYY-MM-DD' (defaults to today)
        """
        if not isinstance(symbol, str):
            raise ValueError("symbol must be a string ticker, e.g., 'AAPL'")
        self.symbol = symbol.upper()
        self.start = start
        self.end = end
        self.data = self.get_data()

    def get_data(self) -> pd.DataFrame:
        """
        Download historical OHLCV data via yfinance and store in a pandas DataFrame.
        - Index set to DatetimeIndex named 'date'
        - Keep a 'close' column (float)
        - Enrich with 'change' and 'instant_return' via calc_returns()
        """
        raw = yf.download(
            self.symbol,
            start=self.start,
            end=self.end,
            interval="1d",
            auto_adjust=False,
            progress=False,
            threads=True,
        )

        if raw is None or raw.empty:
            raise ValueError(f"No data returned for {self.symbol} between {self.start} and {self.end}.")

        data = raw.copy()
        data.index = pd.to_datetime(data.index)
        data.index.name = "date"

        # Standardize the close column
        if "Close" in data.columns:
            data = data[["Close"]].rename(columns={"Close": "close"}).astype(float)
        else:
            close_candidates = [c for c in data.columns if c.lower() == "close"]
            if not close_candidates:
                raise ValueError("Downloaded data does not include a 'Close' column.")
            data = data[[close_candidates[0]]].rename(columns={close_candidates[0]: "close"}).astype(float)

        # Enrich with change & instant_return
        self.calc_returns(data)

        self.data = data
        return data

    def calc_returns(self, df: pd.DataFrame) -> None:
        """
        Helper to add:
        - change: close-to-close dollar change vs. prior day
        - instant_return: daily log return, rounded to 4 decimals
          np.log(close).diff().round(4)
        """
        if "close" not in df.columns:
            raise ValueError("DataFrame must contain a 'close' column before calc_returns().")

        df["change"] = df["close"].diff()
        df["instant_return"] = np.log(df["close"]).diff().round(4)

    def plot_return_dist(self, bins: int = 50) -> None:
        """Plot histogram of instantaneous (log) returns."""
        if self.data is None or self.data.empty:
            raise ValueError("No data to plot. Instantiate the object or call get_data first.")

        returns = self.data["instant_return"].dropna()
        if returns.empty:
            raise ValueError("instant_return series is empty; cannot plot distribution.")

        plt.figure(figsize=(10, 6))
        plt.hist(returns.values, bins=bins, edgecolor="black", alpha=0.8)
        plt.title(f"{self.symbol} — Daily Instantaneous Return Distribution", pad=14)
        plt.xlabel("Instantaneous Return (log)")
        plt.ylabel("Frequency")
        plt.tight_layout()
        plt.show()
    def plot_performance(self) -> None:
        """Plot cumulative performance as percent gain/loss from the first close in the sample."""
        if self.data is None or self.data.empty:
        raise ValueError("No data to plot. Instantiate the object or call get_data first.")

        close = self.data["close"].dropna()
        if close.empty:
            raise ValueError("No 'close' prices available for performance plot.")

        # ✅ FIX: use iloc[0] to extract scalar safely
        base = close.iloc[0]

    series = (close / base) - 1.0  # decimal return from baseline

    fig = plt.figure(figsize=(12, 6))
    ax = fig.gca()
    ax.plot(series.index, series.values, linewidth=2)

    ax.set_title(f"{self.symbol} — Performance Since {series.index[0].date().isoformat()}", pad=14)
    ax.set_xlabel("Date")
    ax.set_ylabel("Return (from start)")
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.grid(True, which="major", alpha=0.3)
    fig.tight_layout()
    plt.show()


    def plot_performance(self) -> None:
        """Plot cumulative performance as percent gain/loss from the first close in the sample."""
        if self.data is None or self.data.empty:
            raise ValueError("No data to plot. Instantiate the o


IndentationError: unindent does not match any outer indentation level (<string>, line 90)